In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

### Importo los datos

In [2]:
libro = pd.ExcelFile('Superstore_Dataset.xlsx')

In [3]:
orders = pd.read_excel(libro, sheet_name = 'Orders')
orders.head(4)

,Row ID+O6G3A1:R6,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,State,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Profit
0,1,CA-2019-152156,2019-11-08,2019-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.960,2,41.9136
1,2,CA-2019-152156,2019-11-08,2019-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.940,3,219.5820
2,3,CA-2019-138688,2019-06-12,2019-06-16,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,California,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.620,2,6.8714
3,13,CA-2020-114412,2020-04-15,2020-04-20,Standard Class,AA-10480,Andrew Allen,Consumer,United States,Concord,North Carolina,South,OFF-PA-10002365,Office Supplies,Paper,Xerox 1967,15.552,3,5.4432


### Elimina las columnas categóricas con más de 4 clases

In [4]:
categoricas = orders.select_dtypes(include=['object'])
categoricas.describe()

,Order ID,Ship Mode,Customer ID,Customer Name,Segment,Country,City,State,Region,Product ID,Category,Sub-Category,Product Name
count,5899,5899,5899,5899,5899,5899,5899,5899,5899,5899,5899,5899,5899
unique,3002,4,773,773,3,1,452,49,4,1755,3,17,1742
top,CA-2020-100111,Standard Class,EP-13915,Emily Phan,Consumer,United States,New York City,California,West,TEC-AC-10003832,Office Supplies,Binders,Easy-staple paper
freq,14,3449,27,27,2996,5899,563,1189,1900,15,3568,915,27


Veo que hay que eliminar 5 variables categóricas

In [5]:
orders.shape

(5899, 19)

In [6]:
for i in categoricas.columns:
    if categoricas[i].nunique()<5:
        orders.drop([i],axis=1, inplace=True)
orders.shape

(5899, 14)

In [38]:
categoricas.shape

(5899, 8)

#### Divide los datos en 3 conjuntos: de entrenamiento 80%, validación 5% y test 15%. Usa "sales" como variable de salida

In [9]:
from sklearn.model_selection import train_test_split

In [11]:
x_train, x_val, y_train, y_val = train_test_split( orders.drop('Sales', axis=1),
                                                   orders.Sales, train_size = 0.8, random_state =1234,
                                                   shuffle=True)
x_test, x_val, y_test, y_val = train_test_split( x_val, y_val, train_size = 0.75, random_state =1234,
                                                   shuffle=True)
print(y_train.shape)
print(y_val.shape)
print(y_test.shape)

(4719,)
(295,)
(885,)


Sobre las columnas numéicas: Aplica una imputación con el valor constante 0 y escala los datos de 0 a 1

In [19]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.compose import make_column_selector
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

In [25]:
numeric_cols =  x_train.select_dtypes(exclude=['object','datetime64[ns]']).columns.to_list()
numeric_cols

['Row ID+O6G3A1:R6', 'Quantity', 'Profit']

In [26]:
cat_cols =  x_train.select_dtypes(include=['object']).columns.to_list()
numeric_cols

['Row ID+O6G3A1:R6', 'Quantity', 'Profit']

In [22]:
numerical_transformer = Pipeline( steps = [('imputer', 
                                          SimpleImputer(strategy='constant',
                                                       fill_value = 0)),
                                         ('scaler', MinMaxScaler())])

Sobre las colummnas categóricas: Aplica una imputación con el valor más frecuente, binariza las columnas y elimina las columnas redundantes

In [27]:
categorical_transformer = Pipeline( steps = [('imputer', 
                                          SimpleImputer(strategy='most_frequent')),
                                         ('onehot', OneHotEncoder())])
preprocessor = ColumnTransformer( transformers=[('numerical', numerical_transformer, 
                                                numeric_cols),
                                               ('cat', categorical_transformer, cat_cols)])

In [31]:
x_train_prep = preprocessor.fit_transform(x_train)
x_test_prep = preprocessor.transform(x_test)
x_val_prep = preprocessor.transform(x_val)

ValueError: Found unknown categories ['CA-2020-139661', 'CA-2020-100580', 'US-2020-118598', 'CA-2019-149237', 'CA-2020-152079', 'CA-2020-160934', 'US-2019-121013', 'CA-2020-111269', 'CA-2020-130043', 'CA-2020-121741', 'CA-2019-114895', 'US-2020-126081', 'CA-2019-140774', 'CA-2020-131303', 'CA-2020-120936', 'US-2019-134761', 'CA-2019-113117', 'CA-2019-142398', 'CA-2019-121958', 'CA-2020-109183', 'CA-2019-119683', 'CA-2019-117912', 'CA-2020-107713', 'CA-2020-108287', 'US-2020-152569', 'CA-2020-147291', 'CA-2020-135783', 'CA-2019-105585', 'CA-2019-140928', 'US-2020-120147', 'CA-2019-158869', 'CA-2020-164959', 'CA-2020-140053', 'CA-2019-164924', 'CA-2020-127264', 'CA-2019-118934', 'CA-2020-151911', 'CA-2020-160087', 'US-2020-111241', 'CA-2020-168900', 'CA-2020-107314', 'CA-2020-158407', 'CA-2020-160899', 'CA-2019-149685', 'US-2019-142685', 'US-2019-131674', 'CA-2020-109211', 'CA-2020-117198', 'CA-2020-102659', 'CA-2019-163174', 'CA-2019-168830', 'CA-2020-164098', 'CA-2020-139437', 'CA-2020

In [ ]:
encoded_cat = proprocessor.named_transformers_['cat']['onehot'].get_feature_names(cat_cols)
labels = np.concatenate([numeric_cols, encoded_cat])
datos_train_prep = pd.DataFrame(x_test_prep, cloumns=labels)
datos_train_prep.describe()

In [32]:
datos_val_prep = pd.DataFrame(x_val_prep, columnd=labels)
datos_test_prep.describe()

NameError: name 'x_val_prep' is not defined